In [162]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings, random
warnings.filterwarnings(action='ignore')

import scipy.stats as stats 


from IPython.display import set_matplotlib_formats

mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus =True)
set_matplotlib_formats('retina')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
import warnings 
warnings.filterwarnings('ignore')
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool
from imblearn.over_sampling import SMOTENC

In [163]:

train = pd.read_csv('../data/open/train.csv')
test = pd.read_csv('../data/open//test.csv')

In [164]:
for i in [train, test]:
    i.drop(['index','FLAG_MOBIL'], axis = 1, inplace = True)
    i.fillna('unemployed', inplace = True)
    i['DAYS_BIRTH'] = np.abs(i['DAYS_BIRTH']) / 365
    i['DAYS_EMPLOYED'] = np.abs(i['DAYS_EMPLOYED']) / 365
    i['begin_month'] = np.abs(i['begin_month']) / 365

In [157]:
train.duplicated().sum()

1634

In [158]:
def dup(df,col):
    collist = list(df.columns)
    
    for i in col:
        collist.remove(i)
    
    return(df.duplicated(subset = collist, keep = False).sum())

In [159]:
collist = list(train.columns)

In [165]:
for i in collist:
    print('%-13s' % i, f' 외 중복 데이터 : {dup(train,[i])}건')

gender         외 중복 데이터 : 3159건
car            외 중복 데이터 : 3155건
reality        외 중복 데이터 : 3155건
child_num      외 중복 데이터 : 3155건
income_total   외 중복 데이터 : 3157건
income_type    외 중복 데이터 : 3159건
edu_type       외 중복 데이터 : 3155건
family_type    외 중복 데이터 : 3157건
house_type     외 중복 데이터 : 3155건
DAYS_BIRTH     외 중복 데이터 : 3437건
DAYS_EMPLOYED  외 중복 데이터 : 3155건
work_phone     외 중복 데이터 : 3155건
phone          외 중복 데이터 : 3155건
email          외 중복 데이터 : 3157건
occyp_type     외 중복 데이터 : 3155건
family_size    외 중복 데이터 : 3159건
begin_month    외 중복 데이터 : 20375건
credit         외 중복 데이터 : 4497건


In [130]:
collist.remove('begin_month')

In [166]:
for i in collist:
    print('begin_month', '& %-13s' % i, f' 외 중복 데이터 : {dup(train,[i,"begin_month"])}건')

begin_month & gender         외 중복 데이터 : 20394건
begin_month & car            외 중복 데이터 : 20375건
begin_month & reality        외 중복 데이터 : 20381건
begin_month & child_num      외 중복 데이터 : 20375건
begin_month & income_total   외 중복 데이터 : 20384건
begin_month & income_type    외 중복 데이터 : 20384건
begin_month & edu_type       외 중복 데이터 : 20375건
begin_month & family_type    외 중복 데이터 : 20389건
begin_month & house_type     외 중복 데이터 : 20375건
begin_month & DAYS_BIRTH     외 중복 데이터 : 20805건
begin_month & DAYS_EMPLOYED  외 중복 데이터 : 20375건
begin_month & work_phone     외 중복 데이터 : 20375건
begin_month & phone          외 중복 데이터 : 20375건
begin_month & email          외 중복 데이터 : 20380건
begin_month & occyp_type     외 중복 데이터 : 20375건
begin_month & family_size    외 중복 데이터 : 20380건


ValueError: list.remove(x): x not in list

In [167]:
colidx = {string : i for i,string in enumerate(list(train.columns))}
print(colidx)

{'gender': 0, 'car': 1, 'reality': 2, 'child_num': 3, 'income_total': 4, 'income_type': 5, 'edu_type': 6, 'family_type': 7, 'house_type': 8, 'DAYS_BIRTH': 9, 'DAYS_EMPLOYED': 10, 'work_phone': 11, 'phone': 12, 'email': 13, 'occyp_type': 14, 'family_size': 15, 'begin_month': 16, 'credit': 17}


In [168]:
train['index'] = ''
test['index'] = ''

In [169]:
for i in [train,test]:
    for j in [0,1,2,3,6,7,11]:
        i['index'] +=  i[i.columns[j]].astype(str)

In [170]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

In [171]:
cat_feats = list(train.dtypes[train.dtypes == 'object'].index)

In [172]:
X = train.drop('credit', axis=1)
y = train['credit']

In [173]:
fold=15
seed=86
kfold = StratifiedKFold(n_splits = fold, shuffle = True, random_state = seed)

In [174]:
logloss = []
n_iter = 0

In [175]:
pred = np.zeros((test.shape[0],3))

In [137]:
seedlogloss = []

for i in range(0,100):
    seed = i
    kfold = StratifiedKFold(n_splits = fold, shuffle = True, random_state = seed)
    logloss = []
    pred = np.zeros((test.shape[0],3))

    for train_index, val_index in kfold.split(X,y):

        x_train, x_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        cat = CatBoostClassifier(n_estimators=2000,early_stopping_rounds=200)
        cat.fit(x_train, y_train, eval_set = [(x_train, y_train),(x_val, y_val)], cat_features = cat_feats, silent = True)

        res = log_loss(y_val, cat.predict_proba(x_val))
        logloss.append(res)


    seedlogloss.append(np.mean(logloss))
print(seedlogloss.index(sorted(seedlogloss)[0]))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

KeyboardInterrupt: 

In [176]:
for train_index, val_index in kfold.split(X,y):
    
    print(f'================================================================================================================================\n{n_iter+1}번째 fold\n')
    x_train, x_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    cat = CatBoostClassifier(n_estimators=2000,early_stopping_rounds=200)
    cat.fit(x_train, y_train, eval_set = [(x_train, y_train),(x_val, y_val)], cat_features = cat_feats, verbose = 200)
    n_iter += 1
    
    res = log_loss(y_val, cat.predict_proba(x_val))
    logloss.append(res)
    
    pred += cat.predict_proba(test)/fold
    
    print('\n{0}번째 폴드 val logloss : {1}'.format(n_iter,res))
    print('================================================================================================================================\n\n\n')

print('평균 val logloss', np.mean(logloss))

1번째 fold

Learning rate set to 0.088346


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 1.0487994	test: 1.0486488	test1: 1.0488710	best: 1.0488710 (0)	total: 69ms	remaining: 2m 17s
200:	learn: 0.7482249	test: 0.6957712	test1: 0.7570563	best: 0.7570529 (198)	total: 13.5s	remaining: 2m 1s
400:	learn: 0.7165756	test: 0.6769374	test1: 0.7461555	best: 0.7461356 (399)	total: 27.7s	remaining: 1m 50s
600:	learn: 0.6887911	test: 0.6637136	test1: 0.7409405	best: 0.7409278 (599)	total: 42s	remaining: 1m 37s
800:	learn: 0.6620949	test: 0.6508910	test1: 0.7338494	best: 0.7338494 (800)	total: 56.3s	remaining: 1m 24s
1000:	learn: 0.6353479	test: 0.6381811	test1: 0.7290799	best: 0.7290636 (985)	total: 1m 10s	remaining: 1m 10s
1200:	learn: 0.6131941	test: 0.6279864	test1: 0.7266420	best: 0.7266420 (1200)	total: 1m 24s	remaining: 56.3s
1400:	learn: 0.5886044	test: 0.6171655	test1: 0.7223373	best: 0.7222726 (1396)	total: 1m 39s	remaining: 42.3s
1600:	learn: 0.5664956	test: 0.6083952	test1: 0.7209277	best: 0.7205261 (1549)	total: 1m 53s	remaining: 28.3s


KeyboardInterrupt: 